In [2]:
# Step 1 import libraries and namespaces
 
import pandas as pd
import torch
 
from torch.utils import data
 
# `nn` is an abbreviation for neural networks
 
from torch import nn

In [4]:
#Step 2: Create Dataset
 
#Define a function to generate noisy data
 
def synthetic_data(m, c, num_examples):
 
    """Generate y = mX + bias(c) + noise"""
    
    X = torch.normal(0, 1, (num_examples, len(m)))
    
    y = torch.matmul(X, m) + c
    
    y += torch.normal(0, 0.01, y.shape)
    
    return X, y.reshape((-1, 1))
 

true_m = torch.tensor([2, -3.4])
 
true_c = 4.2
 
features, labels = synthetic_data(true_m, true_c, 1000)

In [5]:
#Step 3: Read dataset and create small batch
 
#define a function to create a data iterator. Input is the features and labels from synthetic data
 
# Output is iterable batched data using torch.utils.data.DataLoader
 
def load_array(data_arrays, batch_size, is_train=True):
    
    """Construct a PyTorch data iterator."""
    
    dataset = data.TensorDataset(*data_arrays)
    
    return data.DataLoader(dataset, batch_size, shuffle=is_train)
 

 
batch_size = 10
 
data_iter = load_array((features, labels), batch_size)
 

 
next(iter(data_iter))

[tensor([[ 0.9599,  0.2716],
         [-0.1660, -0.9827],
         [ 0.3019, -0.4619],
         [-1.0675,  0.7946],
         [-0.7987, -0.4853],
         [ 0.9335,  1.1235],
         [ 2.3684,  0.3852],
         [ 0.3144, -1.4910],
         [ 0.1892,  0.6928],
         [-0.4479,  0.4324]]),
 tensor([[ 5.1864],
         [ 7.2172],
         [ 6.3720],
         [-0.6311],
         [ 4.2494],
         [ 2.2486],
         [ 7.6322],
         [ 9.8803],
         [ 2.2020],
         [ 1.8357]])]

In [6]:
#Step4: Define model &amp;amp;amp; initialization
 
# Create a single layer feed-forward network with 2 inputs and 1 outputs.
 
model = nn.Linear(2, 1)
 
#Initialize model params
 
model.weight.data.normal_(0, 0.01)
 
model.bias.data.fill_(0)

tensor([0.])

In [7]:
#Step 5: Define loss function
# mean squared error loss function
loss = nn.MSELoss()

In [8]:
#Step 6: Define optimization algorithm
# implements a stochastic gradient descent optimization method
optimizer = torch.optim.SGD(model.parameters(), lr=0.03)

In [10]:
 
# Use complete training data for n epochs, iteratively using a minibatch features and corresponding label
 
# For each minibatch:
 
# &amp;nbsp; Compute predictions by calling net(X) and calculate the loss l
 
# &amp;nbsp; Calculate gradients by running the backpropagation
 
# &amp;nbsp; Update the model parameters using optimizer
 
# &amp;nbsp; Compute the loss after each epoch and print it to monitor progress
 
num_epochs = 5
 
for epoch in range(num_epochs):
 
    for X, y in data_iter:
 
        l = loss(model(X) ,y)
 
        optimizer.zero_grad() #sets gradients to zero
    
        l.backward() # back propagation
    
        optimizer.step() # parameter update
 
    l = loss(model(features), labels)
 
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000399
epoch 2, loss 0.000102
epoch 3, loss 0.000104
epoch 4, loss 0.000103
epoch 5, loss 0.000103


In [11]:
#Results
m = model.weight.data
print('error in estimating m:', true_m - m.reshape(true_m.shape))
c = model.bias.data
print('error in estimating c:', true_c - c)

error in estimating m: tensor([ 8.1062e-06, -8.9788e-04])
error in estimating c: tensor([0.0010])
